<a href="https://colab.research.google.com/github/dayoungcho/dacon/blob/main/tft%EC%98%88%EC%B8%A1_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 세팅

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
!sudo apt-get install -y fonts-nanum

In [ ]:
pip install pytimekr

In [ ]:
pip install pytorch_forecasting

In [ ]:
import pandas as pd
import datetime as dt
import itertools
from pytimekr import pytimekr
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor # Changed for consistency
from pytorch_forecasting import TemporalFusionTransformer, QuantileLoss, Baseline
from pytorch_lightning.loggers import TensorBoardLogger
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

# 데이터 불러오기

In [ ]:
electrade_2013 = pd.read_csv('2013_2016 일별 시간대별 연료원별 전력거래량.csv', encoding='cp949')
electrade_2017 = pd.read_csv('2017~2020 일별 시간대별 연료원별 전력거래량.csv', encoding='cp949')
electrade_2021 = pd.read_csv('일별 시간대별 연료원별 거래량_20220430.csv', encoding='cp949')
electrade_2022 = pd.read_csv('한국전력거래소_연료원별 전력거래량_20231031.csv', encoding='cp949')
electrade_2023 = pd.read_csv('한국전력거래소_연료원별 전력거래량_20231231.csv', encoding='cp949')

In [ ]:
# 데이터 병합

electrade_2013.columns = ['거래일', '거래시간', '연료원', '전력거래량']
electrade_2017.columns = ['거래일', '거래시간', '연료원', '전력거래량']
electrade_2021 = electrade_2021[['거래일', '거래시간', '연료원', '전력거래량']]
electrade_2022 = electrade_2022[['거래일', '거래시간', '연료원', '전력거래량(MWh)']]
electrade_2022.columns = ['거래일', '거래시간', '연료원', '전력거래량']
electrade_2023 = electrade_2023[['거래일', '거래시간', '연료원', '전력거래량(MWh)']]
electrade_2023.columns = ['거래일', '거래시간', '연료원', '전력거래량']

electrade = pd.concat([electrade_2013, electrade_2017, electrade_2021, electrade_2022, electrade_2023]).reset_index(drop=True)
# electrade = pd.concat([electrade_2021, electrade_2022, electrade_2023]).reset_index(drop=True)

In [ ]:
# electrade.loc[electrade['거래시간'] == 24, '거래시간'] = 0   # 자정 거래시간이 0이랑 24로 중복 표기되어있음 -> 0으로 통일
# electrade.거래일 = pd.to_datetime(electrade.거래일)

# # Combine date and hour to create a full datetime timestamp for accurate time indexing
# electrade['full_timestamp'] = electrade['거래일'] + pd.to_timedelta(electrade['거래시간'], unit='h')

# # Sort data to ensure correct chronological order within each group (연료원)
# electrade = electrade.sort_values(by=['full_timestamp', '연료원']).reset_index(drop=True)

In [ ]:
electrade

In [ ]:
# 공휴일 리스트

holidays = []
for i in range(2013,2024):
  holidays.append(pytimekr.holidays(i))
holidays = list(itertools.chain(*holidays))

# 시간별, 연료원별 전력거래량(단기 예측)

In [ ]:
electrade_short = pd.concat([electrade_2021, electrade_2022, electrade_2023]).reset_index(drop=True)
electrade_short.replace('유연탄', '석탄', inplace = True)
electrade_short.replace('무연탄', '석탄', inplace = True)
electrade_short = electrade_short.groupby(['거래일','거래시간','연료원']).sum().reset_index()
electrade_short.loc[electrade_short['거래시간'] == 24, '거래시간'] = 0   # 자정 거래시간이 0이랑 24로 중복 표기되어있음 -> 0으로 통일
electrade_short.거래일 = pd.to_datetime(electrade_short.거래일)

# Combine date and hour to create a full datetime timestamp for accurate time indexing
electrade_short['full_timestamp'] = electrade_short['거래일'] + pd.to_timedelta(electrade_short['거래시간'], unit='h')

# Sort data to ensure correct chronological order within each group (연료원)
electrade_short = electrade_short.sort_values(by=['full_timestamp', '연료원']).reset_index(drop=True)
electrade_short

## 시각화

In [ ]:
fig = px.line(electrade_short, x="full_timestamp", y="전력거래량", color='연료원', title='시간별, 연료원별 전력거래량', template = 'plotly_dark')
fig.update_traces(line_width= 0.7)
fig

In [ ]:
electrade['time_idx_numeric'] = electrade.groupby('연료원')['full_timestamp'].transform(lambda x: (x - x.min()).dt.total_seconds() / 3600).astype(int)

electrade['월'] = electrade.거래일.dt.month.astype(str)
electrade['요일'] = electrade.거래일.dt.weekday.astype(str)
electrade['공휴일여부'] = electrade.거래일.isin(holidays)*1
electrade['공휴일여부'] = electrade['공휴일여부'].astype(str)
electrade['거래시간'] = electrade['거래시간'].astype(str)
# Rename columns to match the desired input for TimeSeriesDataSet and general clarity
electrade.rename(columns={
    '거래일': 'date_original', # Keeping original date for context, if needed
    '거래시간': 'hour',
    '연료원': 'fuel_type',
    '전력거래량': 'volume',
    'time_idx_numeric': 'time_idx' # This is the integer time index for TimeSeriesDataSet
}, inplace=True)

# Drop the 'full_timestamp' column as it's no longer needed after creating 'time_idx'
# And reorder columns for consistency
electrade.columns = ['date_original', 'hour', 'fuel_type', 'volume', 'full_timestamp', 'time_idx', 'month', 'day_of_week', 'is_holiday']

## 모델링

In [ ]:
electrade_short['time_idx_numeric'] = electrade_short.groupby('연료원')['full_timestamp'].transform(lambda x: (x - x.min()).dt.total_seconds() / 3600).astype(int)

electrade_short['월'] = electrade_short.거래일.dt.month.astype(str)
electrade_short['요일'] = electrade_short.거래일.dt.weekday.astype(str)
electrade_short['공휴일여부'] = electrade_short.거래일.isin(holidays)*1
electrade_short['공휴일여부'] = electrade_short['공휴일여부'].astype(str)
electrade_short['거래시간'] = electrade_short['거래시간'].astype(str)
# Rename columns to match the desired input for TimeSeriesDataSet and general clarity
electrade_short.rename(columns={
    '거래일': 'date_original', # Keeping original date for context, if needed
    '거래시간': 'hour',
    '연료원': 'fuel_type',
    '전력거래량': 'volume',
    'time_idx_numeric': 'time_idx', # This is the integer time index for TimeSeriesDataSet
    '월': 'month',
    '요일': 'day_of_week',
    '공휴일여부': 'is_holiday'
}, inplace=True)

# Drop the 'full_timestamp' column as it's no longer needed after creating 'time_idx'
# And reorder columns for consistency
# electrade_short.columns = ['date_original', 'hour', 'fuel_type', 'volume', 'full_timestamp', 'time_idx', 'month', 'day_of_week', 'is_holiday']


In [ ]:
electrade_short

In [ ]:
# 모델 설정
max_prediction_length = 24 # 향후 1일(24시간)을 예측 (decoder 길이)
max_encoder_length = 48   # 과거 2일을 참조 (encoder 길이)
training_cutoff = electrade_short.time_idx.max() - max_prediction_length # 학습 데이터 자르는 시점

training = TimeSeriesDataSet(
    electrade_short[lambda x: x.time_idx <= training_cutoff],   # 학습용 데이터만 필터링

    # ---기본 설정---
    time_idx = "time_idx",
    target = "volume",
    group_ids= ["fuel_type"],

    # ---시퀀스 길이 설정---
    min_encoder_length = max_encoder_length//2,     # 최소 참조 기간(가변적일 수 있음)
    max_encoder_length = max_encoder_length,        # 최대 참조 기간
    min_prediction_length = 1,                      # 최소 예측 기간
    max_prediction_length = max_prediction_length,  # 최대 예측 기간

    # ---변수 매핑(중요)---

    # 1. 정적 변수(static)
    static_categoricals = ['fuel_type'],

    # 2. 미래를 아는 변수(known inputs)
    time_varying_known_categoricals = ['hour', 'day_of_week', 'month', 'is_holiday'],
    time_varying_known_reals = ['time_idx'],   # time_idx 여기에 포함시키기

    # 3. 과거만 아는 변수
    # time_varying_unknown_categoricals = [],
    time_varying_unknown_reals = ['volume'],  # 타겟도 포함시켜야함

    # ---전처리 옵션---
    target_normalizer = GroupNormalizer(
        groups=["fuel_type"], transformation="log1p"
    ),
    add_relative_time_idx = True,  # 시퀀스 내 상대적 시간 인덱스 자동 추가
    add_target_scales = True,      # 타겟의 스케일 정보 추가(정규화 복원 시 필요)
    add_encoder_length = True,     # 인코더 길이 정보 추가
)

In [ ]:
batch_size = 64  # GPU 메모리에 맞춰 조절

In [ ]:
# -----------------------------
# 1단계: DataLoader 만들기
# -----------------------------

# 검증 데이터셋(validation set) 분리: 마지막 시간을 기준으로 나눔
validation = TimeSeriesDataSet.from_dataset(
    training, electrade_short, predict=True,
    stop_randomization=True
)

# 학습용 로더
train_dataloader = training.to_dataloader(
    train = True,
    batch_size = batch_size,
    num_workers = 11
)

# 검증용 로더
val_dataloader = validation.to_dataloader(
    train = False,
    batch_size = batch_size * 10,
    num_workers = 11
)

In [ ]:
# baseline
import torch
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

In [ ]:
# -----------------------------
# 2단계: TFT 모델 생성
# -----------------------------

tft = TemporalFusionTransformer.from_dataset(
    training,
    # [학습 파라미터]
    learning_rate = 0.03, # 0.01~0.05 사이로 설정
    hidden_size = 64, # 모델의 크기(데이터가 적으면 16, 많으면 64-128)
    attention_head_size = 1, # 어텐션 헤드 수(보통 1~4)
    dropout = 0.0, # 과적합 방지(0.1~0.3)
    hidden_continuous_size = 8, # 연속형 변수 처리 크기

    # [손실 함수] -> tft는 확률적 예측을 하므로 QuantileLoss 사용
    loss = QuantileLoss(),

    # [로그 및 최적화 설정]
    log_interval = 10,
    optimizer = "Adam",  # # tft 저자가 추천함..아니면 torch.optim.Adam
    reduce_on_plateau_patience = 4, # 성능이 안 오르면 학습률을 줄임
)


print(f"모델 파라미터 개수: {tft.size()/1e3:.1f}k")

In [ ]:
# -----------------------------
# 3단계: 트레이너(Trainer) 설정 및 학습 시작
# -----------------------------

# 과적합 방지: 검증 성능이 10번 연속 좋아지지 않으면 조기 종료
early_stop_callback = EarlyStopping(
    monitor = "val_loss",
    min_delta = 1e-4,
    patience = 5,
    verbose = False,
    mode = "min"
)

lr_logger = pl.callbacks.LearningRateMonitor()  # 학습률 변화 기록

trainer = pl.Trainer(
    max_epochs = 8,   # 최대 30번 학습(얼리스타핑으로 일찍 끝날 수 있음)
    accelerator = "gpu",  # gpu가 있으면 자동 사용, 없으면 cpu 사용
    devices = 1,   # 사용할 장치 개수
    # precision = 'bf16-mixed',
    enable_model_summary = True,
    gradient_clip_val = 0.1, # 학습 안정화를 위해 기울기 자르기
    callbacks = [lr_logger, early_stop_callback]
)

In [ ]:
# 학습 시작

print("학습을 시작합니다...")
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

In [ ]:
#  Best Model 로드
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path, weights_only=False)

In [ ]:
# calculate MAE on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)
(actuals - predictions).abs().mean()

In [ ]:
# 예측 시각화

raw_predictions, a,b, x,y = best_tft.predict(val_dataloader, mode="raw", return_x=True)
forname = best_tft.predict(val_dataloader, mode="prediction", return_index=True)
name = forname[2]['fuel_type']
for i in range(25):
  best_tft.plot_prediction(a, raw_predictions, idx=i, add_loss_to_title=False)
  plt.title(f"{name[i]} 24시간 거래 예측값")
  plt.show()

# 일별 총 전력거래량(시각화만)

In [ ]:
# 일별 총 전력거래량

electrade_agg_daily = electrade.groupby('거래일').sum().reset_index()
electrade_agg_daily.drop(['거래시간','연료원'], inplace=True,axis=1)
electrade_agg_daily

In [ ]:
fig = px.line(electrade_agg_daily, x="거래일", y="전력거래량", title='일별 총 전력거래량', template = 'plotly')
fig.update_traces(line_width= 0.7)
fig

# 일별, 원료별 전력거래량

In [ ]:
# 일별, 원료별 전력거래량

electrade_type_daily = electrade.groupby(['거래일', '연료원']).sum().reset_index()
electrade_type_daily.drop('거래시간', inplace=True,axis=1)
electrade_type_daily.replace('유연탄', '석탄', inplace = True)
electrade_type_daily.replace('무연탄', '석탄', inplace = True)
electrade_type_daily = electrade_type_daily.groupby(['거래일', '연료원']).sum().reset_index()
electrade_type_daily

In [ ]:
fig = px.line(electrade_type_daily, x="거래일", y="전력거래량", color='연료원', title='일별, 연료원별 전력거래량', template = 'plotly')
fig.update_traces(line_width= 0.7)
fig

# ㅇㅇ

In [ ]:
electrade.loc[electrade['거래시간'] == 24, '거래시간'] = 0   # 자정 거래시간이 0이랑 24로 중복 표기되어있음 -> 0으로 통일
electrade.거래일 = pd.to_datetime(electrade.거래일)

# Combine date and hour to create a full datetime timestamp for accurate time indexing
electrade['full_timestamp'] = electrade['거래일'] + pd.to_timedelta(electrade['거래시간'], unit='h')

# Sort data to ensure correct chronological order within each group (연료원)
electrade = electrade.sort_values(by=['연료원', 'full_timestamp']).reset_index(drop=True)

# Create a numerical time index (integer) as required by TimeSeriesDataSet
# This calculates the number of hours from the earliest timestamp within each fuel_type group.
# We use dt.total_seconds() / 3600 to get hours and then convert to int.
electrade['time_idx_numeric'] = electrade.groupby('연료원')['full_timestamp'].transform(lambda x: (x - x.min()).dt.total_seconds() / 3600).astype(int)

electrade['월'] = electrade.거래일.dt.month.astype(str)
electrade['요일'] = electrade.거래일.dt.weekday.astype(str)
electrade['공휴일여부'] = electrade.거래일.isin(holidays)*1
electrade['공휴일여부'] = electrade['공휴일여부'].astype(str)
electrade['거래시간'] = electrade['거래시간'].astype(str)
# Rename columns to match the desired input for TimeSeriesDataSet and general clarity
electrade.rename(columns={
    '거래일': 'date_original', # Keeping original date for context, if needed
    '거래시간': 'hour',
    '연료원': 'fuel_type',
    '전력거래량': 'volume',
    'time_idx_numeric': 'time_idx' # This is the integer time index for TimeSeriesDataSet
}, inplace=True)

# Drop the 'full_timestamp' column as it's no longer needed after creating 'time_idx'
# And reorder columns for consistency
electrade.columns = ['date_original', 'hour', 'fuel_type', 'volume', 'full_timestamp', 'time_idx', 'month', 'day_of_week', 'is_holiday']

electrade

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=use_spl_gen['splym'], y=use_spl_gen['공급량'],mode = 'lines',name='발전용가스'))
fig.add_trace(go.Scatter(x=use_spl_city['splym'], y=use_spl_city['공급량'],mode = 'lines',name='도시가스'))
fig

In [ ]:
# 모델 설정
max_prediction_length = 24 # 향후 7일(168시간)을 예측 (decoder 길이)
max_encoder_length = 48   # 과거 14일을 참조 (encoder 길이)
training_cutoff = electrade.time_idx.max() - max_prediction_length # 학습 데이터 자르는 시점

training = TimeSeriesDataSet(
    electrade[lambda x: x.time_idx <= training_cutoff],   # 학습용 데이터만 필터링

    # ---기본 설정---
    time_idx = "time_idx",
    target = "volume",
    group_ids= ["fuel_type"],

    # ---시퀀스 길이 설정---
    min_encoder_length = max_encoder_length//2,     # 최소 참조 기간(가변적일 수 있음)
    max_encoder_length = max_encoder_length,        # 최대 참조 기간
    min_prediction_length = 1,                      # 최소 예측 기간
    max_prediction_length = max_prediction_length,  # 최대 예측 기간

    # ---변수 매핑(중요)---

    # 1. 정적 변수(static)
    static_categoricals = ['fuel_type'],

    # 2. 미래를 아는 변수(known inputs)
    time_varying_known_categoricals = ['hour', 'day_of_week', 'month', 'is_holiday'],
    time_varying_known_reals = ['time_idx'],   # time_idx 여기에 포함시키기

    # 3. 과거만 아는 변수
    # time_varying_unknown_categoricals = [],
    time_varying_unknown_reals = ['volume'],  # 타겟도 포함시켜야함

    # ---전처리 옵션---
    target_normalizer = GroupNormalizer(
        groups=["fuel_type"], transformation="log1p"
    ),
    add_relative_time_idx = True,  # 시퀀스 내 상대적 시간 인덱스 자동 추가
    add_target_scales = True,      # 타겟의 스케일 정보 추가(정규화 복원 시 필요)
    add_encoder_length = True,     # 인코더 길이 정보 추가
)





In [ ]:
# 실제 사용을 위한 Dataloader 변환
batch_size = 256  # GPU 메모리에 맞춰 조절

In [ ]:
# -----------------------------
# 1단계: DataLoader 만들기
# -----------------------------

# 검증 데이터셋(validation set) 분리: 마지막 시간을 기준으로 나눔
validation = TimeSeriesDataSet.from_dataset(
    training, electrade, predict=True,
    stop_randomization=True
)

# 학습용 로더
train_dataloader = training.to_dataloader(
    train = True,
    batch_size = batch_size,
    num_workers = 11
)

# 검증용 로더
val_dataloader = validation.to_dataloader(
    train = False,
    batch_size = batch_size * 10,
    num_workers = 11
)


In [ ]:
# -----------------------------
# 2단계: TFT 모델 생성
# -----------------------------

tft = TemporalFusionTransformer.from_dataset(
    training,
    # [학습 파라미터]
    learning_rate = 0.03, # 0.01~0.05 사이로 설정
    hidden_size = 64, # 모델의 크기(데이터가 적으면 16, 많으면 64-128)
    attention_head_size = 1, # 어텐션 헤드 수(보통 1~4)
    dropout = 0.0, # 과적합 방지(0.1~0.3)
    hidden_continuous_size = 8, # 연속형 변수 처리 크기

    # [손실 함수] -> tft는 확률적 예측을 하므로 QuantileLoss 사용
    loss = QuantileLoss(),

    # [로그 및 최적화 설정]
    log_interval = 10,
    optimizer = "Adam",  # # tft 저자가 추천함..아니면 torch.optim.Adam
    reduce_on_plateau_patience = 4, # 성능이 안 오르면 학습률을 줄임
)


print(f"모델 파라미터 개수: {tft.size()/1e3:.1f}k")

In [ ]:
# -----------------------------
# 3단계: 트레이너(Trainer) 설정 및 학습 시작
# -----------------------------

# 과적합 방지: 검증 성능이 10번 연속 좋아지지 않으면 조기 종료
early_stop_callback = EarlyStopping(
    monitor = "val_loss",
    min_delta = 1e-4,
    patience = 5,
    verbose = False,
    mode = "min"
)

lr_logger = pl.callbacks.LearningRateMonitor()  # 학습률 변화 기록

trainer = pl.Trainer(
    max_epochs = 8,   # 최대 30번 학습(얼리스타핑으로 일찍 끝날 수 있음)
    accelerator = "gpu",  # gpu가 있으면 자동 사용, 없으면 cpu 사용
    devices = 1,   # 사용할 장치 개수
    # precision = 'bf16-mixed',
    enable_model_summary = True,
    gradient_clip_val = 0.1, # 학습 안정화를 위해 기울기 자르기
    callbacks = [lr_logger, early_stop_callback]
)

In [ ]:
# 학습 시작

print("학습을 시작합니다...")
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

In [ ]:
#  Best Model 로드
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path, weights_only=False)

# 검증 데이터에 대해 예측 수행(mode="prediction"이면 점추정(quantile 중 0.5, 즉 중앙값 반환). mode="quantiles"면 구간추정(모든 quantile 반환))
raw_predictions, x_decoder, x_encoder = best_tft.predict(
    val_dataloader,
    mode = "raw",
    return_x = True
)

# 결과 시각화
best_tft.plot_prediction(x_decoder, raw_predictions, idx=0, add_loss_to_title=True)

In [ ]:
#  Best Model 로드
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path, weights_only=False)

predictions, index = best_tft.predict(val_dataloader, mode='prediction', return_index = True)
results = index.copy()

results['predicted_volume'] = predictions.tolist()
print(results.head())

In [ ]:
lng_preds = results[results['fuel_type'] == 'LNG']
print('LNG 향후 24시간 거래 예측값:', lng_preds.iloc[0]['predicted_volume'])


In [ ]:
import matplotlib.pyplot as plt

target_fuel = "LNG"

best_tft.plot_prediction(
    val_dataloader,
    predictions,
    index = index[index['fuel_type']==target_fuel].index[0],
    add_loss_to_title = True
)
plt.title(f"{target_fuel} 24시간 거래 예측값")
plt.show()